In [15]:
import polars as pl
from src.main import GetTanksReq, fetch_tank_data

req = GetTanksReq(primo_ids={"110009"})

df = await fetch_tank_data(req)
if df is None:
    raise ValueError("df is None")
lf = df.lazy()
lf.collect()

primo_id,scada_id,metric_nice_name,unique_id,uom,timestamp,value,tanksize
str,str,str,object,str,datetime[μs],f64,f64
"""110009""","""110009W1""","""WaterTank1Volume""",00df10ed-9159-4b71-b564-0cb0b14dfa07,"""bbl""",2024-03-06 04:11:30.972475,214.328095,832.309683
"""110009""","""110009W2""","""WaterTank2Volume""",028aa180-e62d-4079-9b7b-8020a093c4cb,"""bbl""",2024-03-06 04:12:19.965126,270.125642,653.644903
"""110009""","""110009W2""","""WaterTank2InchesToESD""",19ac5155-212e-4848-8385-c75bb90fe624,"""bbl""",2024-03-06 04:12:52.161838,201.847233,653.644903
"""110009""","""11000901""","""OilTank1Level""",1ac3dc19-5a49-441a-92ed-64710d48b4f6,"""in""",2024-03-06 04:12:34.255959,82.727023,479.54315
"""110009""","""110009W2""","""WaterTank2Level""",299d6f76-46e8-49bc-b523-673d70d83097,"""in""",2024-03-06 04:12:19.965126,97.245231,653.644903
…,…,…,…,…,…,…,…
"""110009""","""11000902""","""OilTank2Level""",c7105e6f-8a8e-4b3d-872d-ed2072f4a655,"""in""",2024-03-06 04:12:44.646494,59.778672,531.081137
"""110009""","""110009W2""","""WaterTank2TimeUntilESD""",d0819024-ab0d-4065-894d-8234e28a8ad3,"""days""",2024-03-06 04:10:54.038954,3.161176,653.644903
"""110009""","""11000901""","""OilTank1Volume""",d6589a9f-f49b-4304-afcb-909c6cf38ba2,"""bbl""",2024-03-06 04:12:34.255959,229.797286,479.54315


In [16]:
lf = lf.drop("unique_id")
lf.collect()

primo_id,scada_id,metric_nice_name,uom,timestamp,value,tanksize
str,str,str,str,datetime[μs],f64,f64
"""110009""","""110009W1""","""WaterTank1Volume""","""bbl""",2024-03-06 04:11:30.972475,214.328095,832.309683
"""110009""","""110009W2""","""WaterTank2Volume""","""bbl""",2024-03-06 04:12:19.965126,270.125642,653.644903
"""110009""","""110009W2""","""WaterTank2InchesToESD""","""bbl""",2024-03-06 04:12:52.161838,201.847233,653.644903
"""110009""","""11000901""","""OilTank1Level""","""in""",2024-03-06 04:12:34.255959,82.727023,479.54315
"""110009""","""110009W2""","""WaterTank2Level""","""in""",2024-03-06 04:12:19.965126,97.245231,653.644903
…,…,…,…,…,…,…
"""110009""","""11000902""","""OilTank2Level""","""in""",2024-03-06 04:12:44.646494,59.778672,531.081137
"""110009""","""110009W2""","""WaterTank2TimeUntilESD""","""days""",2024-03-06 04:10:54.038954,3.161176,653.644903
"""110009""","""11000901""","""OilTank1Volume""","""bbl""",2024-03-06 04:12:34.255959,229.797286,479.54315


In [17]:
tank_metrics = ["Level", "Volume", "InchesUntilAlarm", "InchesToESD", "TimeUntilESD", "Capacity", "ID"]
tank_types = ["Water", "Oil"]

tank_metrics_str= "|".join(tank_metrics)
tank_types_str = "|".join(tank_types)

pattern = f'^(?<is_ESD>ESD-)?(?<tank_type>{tank_types_str})Tank(?<tank_number>[0-9]*)(?<tank_metric>{tank_metrics_str})'
lf = lf.with_columns(separated_metrics = pl.col("metric_nice_name").str.extract_groups(pattern))
lf.collect()

primo_id,scada_id,metric_nice_name,uom,timestamp,value,tanksize,separated_metrics
str,str,str,str,datetime[μs],f64,f64,struct[4]
"""110009""","""110009W1""","""WaterTank1Volume""","""bbl""",2024-03-06 04:11:30.972475,214.328095,832.309683,"{null,""Water"",""1"",""Volume""}"
"""110009""","""110009W2""","""WaterTank2Volume""","""bbl""",2024-03-06 04:12:19.965126,270.125642,653.644903,"{null,""Water"",""2"",""Volume""}"
"""110009""","""110009W2""","""WaterTank2InchesToESD""","""bbl""",2024-03-06 04:12:52.161838,201.847233,653.644903,"{null,""Water"",""2"",""InchesToESD""}"
"""110009""","""11000901""","""OilTank1Level""","""in""",2024-03-06 04:12:34.255959,82.727023,479.54315,"{null,""Oil"",""1"",""Level""}"
"""110009""","""110009W2""","""WaterTank2Level""","""in""",2024-03-06 04:12:19.965126,97.245231,653.644903,"{null,""Water"",""2"",""Level""}"
…,…,…,…,…,…,…,…
"""110009""","""11000902""","""OilTank2Level""","""in""",2024-03-06 04:12:44.646494,59.778672,531.081137,"{null,""Oil"",""2"",""Level""}"
"""110009""","""110009W2""","""WaterTank2TimeUntilESD""","""days""",2024-03-06 04:10:54.038954,3.161176,653.644903,"{null,""Water"",""2"",""TimeUntilESD""}"
"""110009""","""11000901""","""OilTank1Volume""","""bbl""",2024-03-06 04:12:34.255959,229.797286,479.54315,"{null,""Oil"",""1"",""Volume""}"


In [18]:
lf = lf.unnest("separated_metrics")
lf.collect()

primo_id,scada_id,metric_nice_name,uom,timestamp,value,tanksize,is_ESD,tank_type,tank_number,tank_metric
str,str,str,str,datetime[μs],f64,f64,str,str,str,str
"""110009""","""110009W1""","""WaterTank1Volume""","""bbl""",2024-03-06 04:11:30.972475,214.328095,832.309683,null,"""Water""","""1""","""Volume"""
"""110009""","""110009W2""","""WaterTank2Volume""","""bbl""",2024-03-06 04:12:19.965126,270.125642,653.644903,null,"""Water""","""2""","""Volume"""
"""110009""","""110009W2""","""WaterTank2InchesToESD""","""bbl""",2024-03-06 04:12:52.161838,201.847233,653.644903,null,"""Water""","""2""","""InchesToESD"""
"""110009""","""11000901""","""OilTank1Level""","""in""",2024-03-06 04:12:34.255959,82.727023,479.54315,null,"""Oil""","""1""","""Level"""
"""110009""","""110009W2""","""WaterTank2Level""","""in""",2024-03-06 04:12:19.965126,97.245231,653.644903,null,"""Water""","""2""","""Level"""
…,…,…,…,…,…,…,…,…,…,…
"""110009""","""11000902""","""OilTank2Level""","""in""",2024-03-06 04:12:44.646494,59.778672,531.081137,null,"""Oil""","""2""","""Level"""
"""110009""","""110009W2""","""WaterTank2TimeUntilESD""","""days""",2024-03-06 04:10:54.038954,3.161176,653.644903,null,"""Water""","""2""","""TimeUntilESD"""
"""110009""","""11000901""","""OilTank1Volume""","""bbl""",2024-03-06 04:12:34.255959,229.797286,479.54315,null,"""Oil""","""1""","""Volume"""


In [19]:
lf = lf.with_columns(pl.col("tank_number").cast(pl.UInt8, strict=False))
lf.collect()

primo_id,scada_id,metric_nice_name,uom,timestamp,value,tanksize,is_ESD,tank_type,tank_number,tank_metric
str,str,str,str,datetime[μs],f64,f64,str,str,u8,str
"""110009""","""110009W1""","""WaterTank1Volume""","""bbl""",2024-03-06 04:11:30.972475,214.328095,832.309683,null,"""Water""",1,"""Volume"""
"""110009""","""110009W2""","""WaterTank2Volume""","""bbl""",2024-03-06 04:12:19.965126,270.125642,653.644903,null,"""Water""",2,"""Volume"""
"""110009""","""110009W2""","""WaterTank2InchesToESD""","""bbl""",2024-03-06 04:12:52.161838,201.847233,653.644903,null,"""Water""",2,"""InchesToESD"""
"""110009""","""11000901""","""OilTank1Level""","""in""",2024-03-06 04:12:34.255959,82.727023,479.54315,null,"""Oil""",1,"""Level"""
"""110009""","""110009W2""","""WaterTank2Level""","""in""",2024-03-06 04:12:19.965126,97.245231,653.644903,null,"""Water""",2,"""Level"""
…,…,…,…,…,…,…,…,…,…,…
"""110009""","""11000902""","""OilTank2Level""","""in""",2024-03-06 04:12:44.646494,59.778672,531.081137,null,"""Oil""",2,"""Level"""
"""110009""","""110009W2""","""WaterTank2TimeUntilESD""","""days""",2024-03-06 04:10:54.038954,3.161176,653.644903,null,"""Water""",2,"""TimeUntilESD"""
"""110009""","""11000901""","""OilTank1Volume""","""bbl""",2024-03-06 04:12:34.255959,229.797286,479.54315,null,"""Oil""",1,"""Volume"""


In [20]:
#pivoting the data
values = pl.col("value")
columns = pl.col("tank_metric")
pivoted_lf = lf.group_by("primo_id", "tank_type", "tank_number", "scada_id").agg(
    values.filter(columns == metric).first().alias(metric) for metric in tank_metrics
)

pivoted_lf.collect()

primo_id,tank_type,tank_number,scada_id,Level,Volume,InchesUntilAlarm,InchesToESD,TimeUntilESD,Capacity,ID
str,str,u8,str,f64,f64,f64,f64,f64,f64,f64
"""110009""","""Water""",1,"""110009W1""",77.158114,214.328095,null,251.702173,6.658088,null,null
"""110009""","""Oil""",1,"""11000901""",82.727023,229.797286,null,248.584227,6.659328,null,null
"""110009""","""Oil""",2,"""11000902""",59.778672,166.051867,null,176.820281,6.314359,null,null
"""110009""","""Water""",2,"""110009W2""",97.245231,270.125642,null,201.847233,3.161176,null,null


In [21]:
null_condition = pl.col("tank_number").is_null()
not_null_condition = pl.col("tank_number").is_not_null()

null_tanks = pivoted_lf.filter(null_condition)
numbered_tanks = pivoted_lf.filter(not_null_condition)

null_tanks = null_tanks.with_columns(pl.col("ID").alias("tank_number"))
null_tanks = null_tanks.drop("ID")
null_tanks = null_tanks.with_columns(pl.col("tank_number").cast(pl.UInt8, strict=False))

null_tanks.collect()

primo_id,tank_type,tank_number,scada_id,Level,Volume,InchesUntilAlarm,InchesToESD,TimeUntilESD,Capacity
str,str,u8,str,f64,f64,f64,f64,f64,f64


In [22]:
joined_lf = numbered_tanks.join(null_tanks, on=["primo_id", "tank_type", "tank_number"], how="left")
joined_lf.collect()

primo_id,tank_type,tank_number,scada_id,Level,Volume,InchesUntilAlarm,InchesToESD,TimeUntilESD,Capacity,ID,scada_id_right,Level_right,Volume_right,InchesUntilAlarm_right,InchesToESD_right,TimeUntilESD_right,Capacity_right
str,str,u8,str,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64
"""110009""","""Water""",2,"""110009W2""",97.245231,270.125642,null,201.847233,3.161176,null,null,null,null,null,null,null,null,null
"""110009""","""Water""",1,"""110009W1""",77.158114,214.328095,null,251.702173,6.658088,null,null,null,null,null,null,null,null,null
"""110009""","""Oil""",2,"""11000902""",59.778672,166.051867,null,176.820281,6.314359,null,null,null,null,null,null,null,null,null
"""110009""","""Oil""",1,"""11000901""",82.727023,229.797286,null,248.584227,6.659328,null,null,null,null,null,null,null,null,null


In [23]:
final_lf = joined_lf.join(lf, on=["scada_id"], how="left")
final_lf = final_lf.group_by("primo_id", "tank_type", "tank_number").agg(pl.all().first())
final_lf.collect()

primo_id,tank_type,tank_number,scada_id,Level,Volume,InchesUntilAlarm,InchesToESD,TimeUntilESD,Capacity,ID,scada_id_right,Level_right,Volume_right,InchesUntilAlarm_right,InchesToESD_right,TimeUntilESD_right,Capacity_right,primo_id_right,metric_nice_name,uom,timestamp,value,tanksize,is_ESD,tank_type_right,tank_number_right,tank_metric
str,str,u8,str,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,str,str,str,datetime[μs],f64,f64,str,str,u8,str
"""110009""","""Oil""",2,"""11000902""",59.778672,166.051867,null,176.820281,6.314359,null,null,null,null,null,null,null,null,null,"""110009""","""OilTank2TimeUntilESD""","""days""",2024-03-06 04:12:10.970106,6.314359,531.081137,null,"""Oil""",2,"""TimeUntilESD"""
"""110009""","""Water""",1,"""110009W1""",77.158114,214.328095,null,251.702173,6.658088,null,null,null,null,null,null,null,null,null,"""110009""","""WaterTank1Volume""","""bbl""",2024-03-06 04:11:30.972475,214.328095,832.309683,null,"""Water""",1,"""Volume"""
"""110009""","""Oil""",1,"""11000901""",82.727023,229.797286,null,248.584227,6.659328,null,null,null,null,null,null,null,null,null,"""110009""","""OilTank1Level""","""in""",2024-03-06 04:12:34.255959,82.727023,479.54315,null,"""Oil""",1,"""Level"""
"""110009""","""Water""",2,"""110009W2""",97.245231,270.125642,null,201.847233,3.161176,null,null,null,null,null,null,null,null,null,"""110009""","""WaterTank2Volume""","""bbl""",2024-03-06 04:12:19.965126,270.125642,653.644903,null,"""Water""",2,"""Volume"""


In [24]:
final_lf = final_lf.with_columns(
    pl.col("primo_id"),
    pl.col("tank_type"),
    pl.col("tank_number"),
    pl.coalesce(pl.col("Level"), pl.col("Level_right")).alias("Level"),
    pl.coalesce(pl.col("Volume"), pl.col("Volume_right")).alias("Volume"),
    pl.coalesce(pl.col("InchesToESD"), pl.col("InchesUntilAlarm_right")).alias("InchesToESD"),
    pl.coalesce(pl.col("TimeUntilESD"), pl.col("TimeUntilESD_right")).alias("TimeUntilESD"),
    pl.coalesce(pl.col("Capacity"), pl.col("tanksize")).alias("Capacity")
)

required_columns = ["primo_id", "tank_type", "tank_number", "Level", "Volume", "InchesToESD", "TimeUntilESD", "Capacity"]
final_lf = final_lf.select(required_columns)
final_lf.collect()

primo_id,tank_type,tank_number,Level,Volume,InchesToESD,TimeUntilESD,Capacity
str,str,u8,f64,f64,f64,f64,f64
"""110009""","""Oil""",2,59.778672,166.051867,176.820281,6.314359,531.081137
"""110009""","""Oil""",1,82.727023,229.797286,248.584227,6.659328,479.54315
"""110009""","""Water""",2,97.245231,270.125642,201.847233,3.161176,653.644903
"""110009""","""Water""",1,77.158114,214.328095,251.702173,6.658088,832.309683


In [25]:
final_lf = final_lf.sort("primo_id", "tank_type", "tank_number")
final_lf.collect()

primo_id,tank_type,tank_number,Level,Volume,InchesToESD,TimeUntilESD,Capacity
str,str,u8,f64,f64,f64,f64,f64
"""110009""","""Oil""",1,82.727023,229.797286,248.584227,6.659328,479.54315
"""110009""","""Oil""",2,59.778672,166.051867,176.820281,6.314359,531.081137
"""110009""","""Water""",1,77.158114,214.328095,251.702173,6.658088,832.309683
"""110009""","""Water""",2,97.245231,270.125642,201.847233,3.161176,653.644903


In [26]:
percent_tank_full = (pl.col("Volume") / pl.col("Capacity") * 100).round().cast(pl.UInt8)
final_lf = final_lf.with_columns(percent_tank_full.alias("percent_full"))

final_lf.collect()

primo_id,tank_type,tank_number,Level,Volume,InchesToESD,TimeUntilESD,Capacity,percent_full
str,str,u8,f64,f64,f64,f64,f64,u8
"""110009""","""Oil""",1,82.727023,229.797286,248.584227,6.659328,479.54315,48
"""110009""","""Oil""",2,59.778672,166.051867,176.820281,6.314359,531.081137,31
"""110009""","""Water""",1,77.158114,214.328095,251.702173,6.658088,832.309683,26
"""110009""","""Water""",2,97.245231,270.125642,201.847233,3.161176,653.644903,41


In [27]:
capacity_rounded = pl.col("Capacity").round()
final_lf = final_lf.with_columns(capacity_rounded.alias("Capacity"))

final_lf.collect()

primo_id,tank_type,tank_number,Level,Volume,InchesToESD,TimeUntilESD,Capacity,percent_full
str,str,u8,f64,f64,f64,f64,f64,u8
"""110009""","""Oil""",1,82.727023,229.797286,248.584227,6.659328,480.0,48
"""110009""","""Oil""",2,59.778672,166.051867,176.820281,6.314359,531.0,31
"""110009""","""Water""",1,77.158114,214.328095,251.702173,6.658088,832.0,26
"""110009""","""Water""",2,97.245231,270.125642,201.847233,3.161176,654.0,41


In [28]:
volume_to_feet = pl.col("Volume").round().cast(pl.UInt64)

final_lf = final_lf.with_columns(volume_to_feet.alias("Volume"))

final_lf.collect()

primo_id,tank_type,tank_number,Level,Volume,InchesToESD,TimeUntilESD,Capacity,percent_full
str,str,u8,f64,u64,f64,f64,f64,u8
"""110009""","""Oil""",1,82.727023,230,248.584227,6.659328,480.0,48
"""110009""","""Oil""",2,59.778672,166,176.820281,6.314359,531.0,31
"""110009""","""Water""",1,77.158114,214,251.702173,6.658088,832.0,26
"""110009""","""Water""",2,97.245231,270,201.847233,3.161176,654.0,41
